# Task 2A: Recommendation What Family Family Would Buy Next

# I. Imports

In [2]:
import pandas as pd
import pandas as pd
import numpy as np
import surprise
import apyori
from apyori import apriori
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Загружаем приготовленный датасет
df_village = pd.read_csv('village_table.csv')

# II. Transformation

In [4]:
data = df_village.copy()

In [5]:
data.head()

,village,year,family,category,fruit,grade,order_date,delivery_date,ordered,shipped,leadtime,leadtime_int,order_date_int
0,Robinhood_village,2020,family_244,category_25,fruit_37,165,2019-12-30,2020-01-03,1650.0,1663.20,4 days,4,1577664000
1,Robinhood_village,2020,family_127,category_14,fruit_62,250,2019-12-30,2020-01-10,1000.0,1060.00,11 days,11,1577664000
2,Robinhood_village,2020,family_133,category_16,fruit_48,330,2019-12-30,2020-01-08,1320.0,1336.50,9 days,9,1577664000
3,Robinhood_village,2020,family_133,category_25,fruit_49,250,2019-12-30,2020-01-08,2500.0,2508.75,9 days,9,1577664000
4,Robinhood_village,2020,family_133,category_4,fruit_52,315,2019-12-30,2020-01-08,4725.0,4759.65,9 days,9,1577664000


In [6]:
data.order_date.max()

'2022-02-20'

In [7]:
train = data[data.order_date < '2022-02-12']
val = data[data.order_date >= '2022-02-12']

In [8]:
train = train[['family','fruit','ordered']]
val = val[['family','fruit','ordered']]

In [9]:
# Данные по заказам, которые нужно суммировать
train = train.groupby(['family','fruit']).agg(['count'])
train.reset_index(inplace=True)
train.columns = train.columns.droplevel(1)
train.columns = ['family','fruit','order_count']

val = val.groupby(['family','fruit']).agg(['count'])
val.reset_index(inplace=True)
val.columns = val.columns.droplevel(1)
val.columns = ['family','fruit','order_count']

In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18346 entries, 0 to 18345
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   family       18346 non-null  object
 1   fruit        18346 non-null  object
 2   order_count  18346 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 430.1+ KB


In [11]:
val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 644 entries, 0 to 643
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   family       644 non-null    object
 1   fruit        644 non-null    object
 2   order_count  644 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 15.2+ KB


In [12]:
train['order_count'].describe()

count    18346.000000
mean        13.240434
std         39.971358
min          1.000000
25%          1.000000
50%          4.000000
75%         11.000000
max       1531.000000
Name: order_count, dtype: float64

In [13]:
# Создаем перечень часто покупаемых фруктов семьями в обучающей выборке
train_popular = train[train.order_count > 10].groupby('family')['fruit']
# В виде строки разделенных пробелом фруктов
train_pop_str = train_popular.apply(lambda r: ' '.join([str(A) for A in r]))
# Выведем список частопокупаемых продуктов
train_family_list = list(train_pop_str.index)
# Таблица по семьям в виде вложенного в ячейку списка
train_pop_list = train[train.order_count > 11].groupby('family')['fruit'].apply(list)

In [14]:
train_pop_list

family
family_0      [fruit_4470, fruit_4471, fruit_4472, fruit_447...
family_1      [fruit_133, fruit_136, fruit_138, fruit_180, f...
family_10                                 [fruit_57, fruit_883]
family_100    [fruit_1020, fruit_1286, fruit_136, fruit_145,...
family_101    [fruit_1149, fruit_182, fruit_189, fruit_345, ...
                                    ...                        
family_95     [fruit_1168, fruit_1283, fruit_136, fruit_154,...
family_96     [fruit_1113, fruit_124, fruit_132, fruit_1424,...
family_97     [fruit_1068, fruit_1135, fruit_1149, fruit_115...
family_98                               [fruit_1478, fruit_917]
family_99     [fruit_1118, fruit_1119, fruit_1222, fruit_124...
Name: fruit, Length: 273, dtype: object

In [15]:
# Создаем перечень купленных фруктов семьями в валидационной выборке
val_grouped = val.groupby('family')['fruit']
val_grouped = val_grouped.apply(lambda r: ' '.join([str(A) for A in r]))
# Выведем список купленных фруктов в обучающей выборке
val_family_list = list(val_grouped.index)
# Таблица по семьям в виде вложенного в ячейку списка
val_list = val.groupby('family')['fruit'].apply(list)

In [16]:
val_grouped

family
family_0      fruit_1517 fruit_2853 fruit_4478 fruit_4481 fr...
family_100                     fruit_1202 fruit_3117 fruit_4169
family_101                                           fruit_1149
family_102                                           fruit_3568
family_105                     fruit_1318 fruit_2013 fruit_2599
                                    ...                        
family_92                                            fruit_3455
family_93                                   fruit_189 fruit_621
family_96     fruit_1113 fruit_143 fruit_1567 fruit_171 frui...
family_97     fruit_1774 fruit_1862 fruit_2209 fruit_3 fruit...
family_99     fruit_1118 fruit_1222 fruit_155 fruit_2472 fru...
Name: fruit, Length: 128, dtype: object

In [17]:
# Перечень семей в обучающей выборке
len(train_family_list)

277

In [18]:
# Перечень семей в валидационной выборке
len(val_family_list)

128

In [19]:
# Семьи, которые не покупали в период, выбранный для обучающей выборки
list(set(val_family_list) - set(train_family_list))

['family_379',
 'family_236',
 'family_161',
 'family_322',
 'family_395',
 'family_292',
 'family_399']

In [276]:
val[val.family == 'family_399'].family.iloc[0] not in train_family_list
# not in train_family_list

True

In [277]:
# Выводим строку часто покупаемых фруктов для семьи 100
train_pop_str['family_100']

'fruit_1020 fruit_1286 fruit_136 fruit_145 fruit_176 fruit_1776 fruit_19 fruit_2085 fruit_252 fruit_283 fruit_29 fruit_2926 fruit_2991 fruit_331 fruit_3329 fruit_3518 fruit_56 fruit_57'

### Ranking 

Выведем ранкинг наиболее часто покупаемых фруктов в деревне:

In [278]:
train_total_rank = train.copy()
train_total_rank = train_total_rank.groupby(['fruit']).sum()
train_total_rank = train_total_rank.sort_values(['order_count'], ascending=False)

In [279]:
train_total_rank.head(3)

,order_count
fruit,
fruit_19,7912
fruit_59,7585
fruit_189,6359


In [280]:
train_total_top_3 = list(train_total_rank.index)[:3]

In [281]:
train_total_top_5 = list(train_total_rank.index)[:5]

Для более детального прогнозирования можно вывести наиболее популярных фруктов для каждой семьи в обучающей выборке

In [282]:
train_rank = train.copy()
train_rank = train_rank.sort_values(['family','order_count'], ascending=False)
train_rank = train_rank.groupby(['family']).first()

train_rank_2 = train.copy()
train_rank_2 = train_rank_2.sort_values(['family','order_count'], ascending=False)
train_rank_2 = train_rank_2.groupby(['family']).nth(1)

train_rank_3 = train.copy()
train_rank_3 = train_rank_3.sort_values(['family','order_count'], ascending=False)
train_rank_3 = train_rank_3.groupby(['family']).nth(2)

train_frames = [train_rank, train_rank_2, train_rank_3]
train_top_3_ranking = pd.concat(train_frames)
train_top_3_ranking.reset_index(inplace=True)

train_ranking = train_top_3_ranking.groupby('family')['fruit']
train_ranking = train_ranking.apply(lambda r: ' '.join([str(A) for A in r]))

train_top_list = train_top_3_ranking.groupby('family')['fruit'].apply(list)
train_top_df = train_top_list.reset_index()
train_top_df
train_top_df.columns = ['family','top_3']

Альтернативный вариант для 5 наиболее покупаемых продуктах

In [283]:
train_rank = train.copy()
train_rank = train_rank.sort_values(['family','order_count'], ascending=False)
train_rank = train_rank.groupby(['family']).first()

train_rank_2 = train.copy()
train_rank_2 = train_rank_2.sort_values(['family','order_count'], ascending=False)
train_rank_2 = train_rank_2.groupby(['family']).nth(1)

train_rank_3 = train.copy()
train_rank_3 = train_rank_3.sort_values(['family','order_count'], ascending=False)
train_rank_3 = train_rank_3.groupby(['family']).nth(2)

train_rank_4 = train.copy()
train_rank_4 = train_rank_4.sort_values(['family','order_count'], ascending=False)
train_rank_4 = train_rank_4.groupby(['family']).nth(3)

train_rank_5 = train.copy()
train_rank_5 = train_rank_5.sort_values(['family','order_count'], ascending=False)
train_rank_5 = train_rank_5.groupby(['family']).nth(3)

train_frames = [train_rank, train_rank_2, train_rank_3, train_rank_4, train_rank_5]
train_top_5_ranking = pd.concat(train_frames)
train_top_5_ranking.reset_index(inplace=True)

train_ranking = train_top_5_ranking.groupby('family')['fruit']
train_ranking = train_ranking.apply(lambda r: ' '.join([str(A) for A in r]))

train_top_list = train_top_5_ranking.groupby('family')['fruit'].apply(list)
train_top_df = train_top_list.reset_index()
train_top_df
train_top_df.columns = ['family','top']

Список купленных фруктов в валидационной выборке будет служить для определения точности предсказания

In [285]:
val_grouped

family
family_0      fruit_1517 fruit_2853 fruit_4478 fruit_4481 fr...
family_100                     fruit_1202 fruit_3117 fruit_4169
family_101                                           fruit_1149
family_102                                           fruit_3568
family_105                     fruit_1318 fruit_2013 fruit_2599
                                    ...                        
family_92                                            fruit_3455
family_93                                   fruit_189 fruit_621
family_96     fruit_1113 fruit_143 fruit_1567 fruit_171 frui...
family_97     fruit_1774 fruit_1862 fruit_2209 fruit_3 fruit...
family_99     fruit_1118 fruit_1222 fruit_155 fruit_2472 fru...
Name: fruit, Length: 128, dtype: object

In [286]:
val_list

family
family_0      [fruit_1517, fruit_2853, fruit_4478, fruit_448...
family_100                 [fruit_1202, fruit_3117, fruit_4169]
family_101                                         [fruit_1149]
family_102                                         [fruit_3568]
family_105                 [fruit_1318, fruit_2013, fruit_2599]
                                    ...                        
family_92                                          [fruit_3455]
family_93                                [fruit_189, fruit_621]
family_96     [fruit_1113, fruit_143, fruit_1567, fruit_171,...
family_97     [fruit_1774, fruit_1862, fruit_2209, fruit_3, ...
family_99     [fruit_1118, fruit_1222, fruit_155, fruit_2472...
Name: fruit, Length: 128, dtype: object

# III. Creating Recomendation Rules

### Method 1

In [313]:
# Создаем ассоциативные правила
association_rules = apriori(train_pop_str.apply(lambda r: r.split(' ')), 
                                   min_support=0.022, 
                                   min_confidence=0.1, 
                                   min_lift=3, 
                                   min_length=1)

association_result = list(association_rules)

Посмотрим на количество созданных правил

In [314]:
len(association_result)

10687

Ассоциативые правила необходимо преобразовать в таблицу рекомендаций, чтобы правила были более удобными для прочиеня

In [319]:
results = []
for item in association_result:
    pair = item[0] 
    items = [x for x in pair]
    value_0 = str(items[0])
    value_1 = str(items[1])
    value_2 = str(item[1])[:7]
    value_3 = str(item[2][0][2])[:7]
    value_4 = str(item[2][0][3])[:7]
    
    rows = value_0, value_1, value_2, value_3, value_4

    results.append(rows)
    label = ['title_1', 'title_2', 'support', 'confidence','lift']
    
    store_suggestion = pd.DataFrame.from_records(results, columns=label)

Для какого количества фруктов созданы рекомендации:

In [316]:
len(store_suggestion.title_1.value_counts())

79

Всего в валидационной выборке уникальных фруктов

In [317]:
val.fruit.nunique()

494

In [324]:
store_suggestion = store_suggestion.sort_values(['lift','confidence'], ascending=False).reset_index()
store_suggestion.head(10)

,level_0,index,title_1,title_2,support,confidence,lift
0,0,459,fruit_123,fruit_154,0.02527,0.53846,9.94358
1,1,372,fruit_407,fruit_381,0.02527,0.53846,9.94358
2,3,1638,fruit_59,fruit_123,0.02527,0.53846,9.94358
3,4,439,fruit_124,fruit_123,0.02527,0.53846,9.94358
4,6,483,fruit_59,fruit_123,0.02527,0.53846,9.94358
5,2,364,fruit_345,fruit_381,0.02527,0.46666,9.94358
6,5,340,fruit_399,fruit_31,0.02527,0.46666,9.94358
7,7,1145,fruit_39,fruit_2150,0.02527,0.38888,9.79292
8,8,1472,fruit_59,fruit_323,0.02527,0.35,9.695
9,9,3601,fruit_193,fruit_2366,0.02527,0.27999,9.69499


### Предсказания

Создаем датафрейм с предсказаниями на основе рекоммендации априори. 
* Выбираем правила на основании фруктов, которые семья относительно часто покупала до этого (train_pop_str). 
* В случае, если в ассоцитивых правилах не было фруктов, которые покупала семья, предлагаем семье фрукты, которые ранее семья чаще всего покупала. 
* Есть также семьи, которые не покупали в период, который был выбран за обучающую выборку. Для таких семей можно порекомендовать наиболее популярные продукты во всей деревне (train_total_top_3)

In [294]:
train_top_df.head()

,family,top
0,family_0,"[fruit_4478, fruit_4482, fruit_4481, fruit_449..."
1,family_1,"[fruit_663, fruit_182, fruit_133, fruit_136, f..."
2,family_10,"[fruit_57, fruit_883, fruit_478, fruit_730, fr..."
3,family_100,"[fruit_1776, fruit_176, fruit_19, fruit_252, f..."
4,family_101,"[fruit_182, fruit_59, fruit_189, fruit_60, fru..."


In [295]:
train_total_top_5

['fruit_19', 'fruit_59', 'fruit_189', 'fruit_124', 'fruit_133']

In [296]:
# Семьи, которые не покупали в периоде обучающей выбоки. Т.е. новые клиенты
missing = list(set(val_family_list) - set(train_family_list))

In [297]:
missing

['family_399',
 'family_292',
 'family_161',
 'family_379',
 'family_395',
 'family_322',
 'family_236']

Создание датафрейма предсказаний

In [331]:
missing = list(set(val_family_list) - set(train_family_list))

result = pd.DataFrame(columns = ['family','recommended'])
for family in val_family_list:

    element = []
    x = 0
    y = 0
    list_1 = train_top_list.loc[family]
    list_2 = store_suggestion['title_1'].unique()
    common = list(set(list_1).intersection(list_2))
    
    if family in missing:
        for l in train_total_top_3:
            element.append(l)
        new_row = {'family':family, 'recommended':element}

    elif len(common) > 0:
        for i in store_suggestion.index:
            if store_suggestion['title_1'].loc[i].split(' ')[0] in train_pop_str.loc[family]:
                if store_suggestion['title_2'].loc[i] not in element:
                    element.append(store_suggestion['title_2'].loc[i])
                    x += 1
                    if x >= 3:
                        break
                else:
                    continue
        new_row = {'family':family, 'recommended':element}
    else:
        for j in train_top_df[train_top_df.family == family].top:
            for k in j:
                element.append(k)
        new_row = {'family':family, 'recommended':element}       
        
    result = result.append(new_row,ignore_index=True)

Альтернативный вариант

In [329]:
missing = list(set(val_family_list) - set(train_family_list))

result = pd.DataFrame(columns = ['family','recommended'])
for family in val_family_list:

    element = []
    x = 0
    y = 0
    list_1 = train_top_list.loc[family]
    list_2 = store_suggestion['title_1'].unique()
    common = list(set(list_1).intersection(list_2))
    
    if family in missing:
        for l in train_total_top_5:
            element.append(l)
        new_row = {'family':family, 'recommended':element}

    elif len(common) > 0:
        for i in store_suggestion.index:
            if store_suggestion['title_1'].loc[i].split(' ')[0] in train_pop_str.loc[family]:
                if store_suggestion['title_2'].loc[i] not in element:
                    element.append(store_suggestion['title_2'].loc[i])
                    x += 1
                    if x >= 3:
                        break
                else:
                    continue
        new_row = {'family':family, 'recommended':element}
    else:
        for j in train_top_df[train_top_df.family == family].top:
            for k in j:
                element.append(k)
        new_row = {'family':family, 'recommended':element}       
        
    result = result.append(new_row,ignore_index=True)

Альтернативный вариант 2

In [327]:
missing = list(set(val_family_list) - set(train_family_list))

result = pd.DataFrame(columns = ['family','recommended'])
for family in val_family_list:

    element = []
    x = 0
    y = 0
    list_1 = train_top_list.loc[family]
    list_2 = store_suggestion['title_1'].unique()
    common = list(set(list_1).intersection(list_2))
    
    if family in missing:
        for l in train_total_top_5:
            element.append(l)
        new_row = {'family':family, 'recommended':element}

    elif len(common) > 0:
        for i in store_suggestion.index:
            if store_suggestion['title_1'].loc[i].split(' ')[0] in train_pop_str.loc[family]:
                if store_suggestion['title_2'].loc[i] not in element:
                    element.append(store_suggestion['title_2'].loc[i])
                    x += 1
                    if x >= 10:
                        break
                else:
                    continue
        new_row = {'family':family, 'recommended':element}
    else:
        for j in train_top_df[train_top_df.family == family].top:
            for k in j:
                element.append(k)
        new_row = {'family':family, 'recommended':element}       
        
    result = result.append(new_row,ignore_index=True)

In [300]:
# Проверяем результат
result[result.family == 'family_101']

,family,recommended
2,family_101,"[fruit_124, fruit_19, fruit_2366]"


### Evaluation

In [332]:
evaluation = pd.DataFrame(columns = ['family','fruit','eval'])

true_positive = 0
false_positive = 0
for fam in val_family_list:
    for i in result[result.family == fam]['recommended'].iloc[0]:
        if i in val_df[val_df.family == fam].true_top.iloc[0]:
            true_positive += 1
        else:
            false_positive += 1
            
precision = true_positive/(true_positive+false_positive)
print('true positive = ', true_positive)
print('false positive = ', false_positive)
print('precision = ', precision)


true positive =  49
false positive =  382
precision =  0.1136890951276102


### Alternative Method Association Rules #2

In [ ]:
# Создаем ассоциативные правила
association_rules_2 = apyori.apriori(data_pop_str.apply(lambda r: r.split(' ')), 
                                   min_support=0.04, 
                                   min_confidence=0.1, 
                                   min_lift=2, 
                                   min_length=2)

In [ ]:
# Функция для вывода рекомендации на экран
asr_df = pd.DataFrame(columns = ['from', 'to', 'confidence', 'support', 'lift'])
for item in association_rules_2:
    pair = item[0] 
    items = [x for x in pair]
    asr_df.loc[len(asr_df), :] =  ' '.join(list(item[2][0][0])), \
                                  ' '.join(list(item[2][0][1])),\
                                  item[2][0][2], item[1], item[2][0][3]

In [ ]:
len(asr_df)

In [ ]:
asr_df.sort_values(['support','confidence'], ascending=False).head(10)

In [ ]:
asr_df.sort_values(['to'], ascending=False).head(10)

In [ ]:
asr_df.iloc[0]

In [ ]:
asr_df

### Alternative Method #3

In [ ]:
# !pip install mlxtend  

In [ ]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [ ]:
df = data.copy()
df = df[data.order_count > 11].groupby('family')['fruit'].apply(list)

test = df
oht = TransactionEncoder()
oht_ary = oht.fit(test).transform(test)
d = pd.DataFrame(oht_ary,columns=oht.columns_)
r_test = apriori(d,min_support=0.001, use_colnames=True, max_len = 2).sort_values(by='support', ascending=False)

In [ ]:
res_test = association_rules(r_test, min_threshold=0.0001).sort_values(['antecedent support','confidence'], ascending=False)

In [ ]:
res_test